#### Tensors

In [1]:
# Create an array in numpy

import numpy as np
np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [2]:
# Creating a tensor in pytorch

import torch

torch.Tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

#### Automatic differentiation
- Numpy natively doesnot offer automatic differentiation and optimization
- PyTorch inherently embeds these features into their data structures or tensors

In [3]:
numIn = 3
numHidden = 1
numOut = 1

# Example Dataset
X = torch.Tensor([[0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1], [0, 1, 1], [0, 1, 0]])
print('X ({}):\n{}'.format(X.shape, X))
y = torch.Tensor([[1], [0], [0], [1], [1], [0], [1]])
print('y ({}):\n{}'.format(y.shape, y))

# Generate the random weights
weights = torch.rand((numIn, numHidden), requires_grad=True).T
print('weights ({}):\n{}'.format(weights.shape, weights))

# Generate the random bias 
bias = torch.rand(numHidden, requires_grad=True)
print('bias ({}):{}'.format(bias.shape, bias))

X (torch.Size([7, 3])):
tensor([[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.],
        [0., 1., 1.],
        [0., 1., 0.]])
y (torch.Size([7, 1])):
tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]])
weights (torch.Size([1, 3])):
tensor([[0.3540, 0.2952, 0.4173]], grad_fn=<PermuteBackward0>)
bias (torch.Size([1])):tensor([0.4891], requires_grad=True)


In [4]:
yhat = weights*X+bias #linear model
loss = (torch.sum(y - yhat))**2 #loss function
loss.backward() #calculates gradients

In [6]:
yhat.shape

torch.Size([7, 3])

In [7]:
X.shape

torch.Size([7, 3])

In [8]:
weights.shape

torch.Size([1, 3])

In [9]:
bias.shape

torch.Size([1])

In [10]:
y.shape

torch.Size([7, 1])

In [5]:
# Print the gradients
print('weight: ', weights.grad)
print('bias: ', bias.grad)

weight:  None
bias:  tensor([100.3023])


C:\Users\chakr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten\src\ATen/core/TensorBody.h:475.)
  return self._grad


#### Building Models
- letting tensors hold their gradient, allows PyTorch's **nn** module to automatically utilize it
  - for setting up feedforward neural network models
  - use the built-in optimization algorithms
  - automatically train the networks
  - without manual specification of back propogation and gradient decent

In [6]:
# Creating a simple model
model = torch.nn.Sequential(
    torch.nn.Linear(numIn, numHidden),
    torch.nn.Sigmoid(),
    torch.nn.Linear(numHidden, numOut)
)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [7]:
# Train the model
numSteps = 25000
for step in range(numSteps):
    # Forward Pass
    y_pred = model(X)
    # Calculate the cost
    loss = loss_fn(y_pred, y)
    # Set the grads to None
    optimizer.zero_grad()
    # Backward Pass
    loss.backward()
    # Updates the parameters
    optimizer.step()

    # Print the progress
    if step % np.round(numSteps/10) == 0:               
        print('epoch: {:d}, error: {:.2f}'.format(step, loss))

epoch: 0, error: 1.42
epoch: 2500, error: 0.06
epoch: 5000, error: 0.04
epoch: 7500, error: 0.03
epoch: 10000, error: 0.02
epoch: 12500, error: 0.02
epoch: 15000, error: 0.02
epoch: 17500, error: 0.01
epoch: 20000, error: 0.01
epoch: 22500, error: 0.01


In [8]:
# Displaying the results
import pandas as pd
y_pred = model(X)
print('actual:\n', y)
print('predicted:\n', torch.round(y_pred))

actual:
 tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]])
predicted:
 tensor([[1.],
        [-0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]], grad_fn=<RoundBackward0>)


In [9]:
# Testing on unseen data
# Test 1
X_test = torch.Tensor([[1, 0, 0], [0,1,0]])
result = model(X_test)
print('result:\n{}'.format(torch.round(result)))

result:
tensor([[0.],
        [1.]], grad_fn=<RoundBackward0>)
